In [1]:
from src import util
from src import plotting

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# %matplotlib qt
%matplotlib qt



2023-08-14 09:07:48.367 Python[32145:1305744] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [2]:

def plot_trj_3d(trjs, labels, legend=False, savename=None):

    fig, ax = plt.subplots(subplot_kw=dict(projection="3d"))
    
    lines = []

    for trj, label in zip(trjs,labels):
    
        line, = ax.plot(trj[:,0],trj[:,1],trj[:,2], label=label)
        ax.scatter(trj[0,0], trj[0,1], trj[0,2], marker='o', color="black")
        ax.scatter(trj[-1,0], trj[-1,1], trj[-1,2], marker='x', color='red')
        lines.append(line)
    
    start_marker = mlines.Line2D([], [], marker='o', color="black", linestyle='None',
                          markersize=5, label='Start')
    stop_marker = mlines.Line2D([], [], marker='x', color='red', linestyle='None',
                          markersize=5, label='Stop')


    lines += [start_marker, stop_marker]

    plt.legend(handles = lines)
        
    if savename:
        plt.savefig("plots/" + savename + ".pdf")
        
    plt.show(block=False)
    plt.pause(0.001)

In [33]:
import numpy as np
import matplotlib.pyplot as plt

def generate_parabola_trajectory(start_point, end_point, duration, num_points):
    t = np.linspace(0, duration, num=num_points)  # Time vector

    # Desired velocity profile (parabola)
    v_max = np.linalg.norm(end_point - start_point) / duration  # Maximum velocity
    v_desired = 2 * v_max * (1 - (t / duration - 0.5)**2)

    # Integrate velocity profile to get positions using numerical integration
    dt = duration / (num_points - 1)
    z = np.zeros(num_points)
    for i in range(1, num_points):
        z[i] = z[i - 1] + (v_desired[i - 1] + v_desired[i]) * 0.5 * dt

    # Normalize the z values to span from start to end points
    z = start_point[2] + (z - z[0]) * (end_point[2] - start_point[2]) / (z[-1] - z[0])

    # Generate x and y coordinates
    x = np.linspace(start_point[0], end_point[0], num=num_points)  # Linear x movement
    y = np.linspace(start_point[1], end_point[1], num=num_points)  # Linear y movement

    trajectory = np.column_stack((x, y, z))
    return trajectory

start_point = np.array([-1, -1, 0])
end_point = np.array([2, 2, 0])
duration = 5
num_points = 100

trajectory = generate_parabola_trajectory(start_point, end_point, duration, num_points)

# Plotting the trajectory
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Parabolic Trajectory')
plt.show()


In [30]:
def gen3d_trj(startpoint, endpoint, duration=5, dt=0.1):


    center = duration/2
    lower_bound = 0
    upper_bound = duration
    N = duration/dt
    lower_bound_to_center = lower_bound + np.geomspace(lower_bound,center, num=int(N/2))
    upper_bound_to_center = upper_bound - np.geomspace(center,upper_bound, num=int(N/2))
    ts = list(lower_bound_to_center) + list(upper_bound_to_center)

    gamma = [startpoint]

    t = dt

    e_t = (endpoint-startpoint)

    for t in ts[1:]:

        point_at_t = startpoint + t / duration * e_t

        gamma.append(point_at_t)

    gamma.append(endpoint)

    return np.array(gamma)

In [31]:
trj = gen3d_trj(np.array([1,1,0]), np.array([-0.5, -0.5, 0]))

fig, ax = plt.subplots(subplot_kw=dict(projection="3d"))

ax.scatter(trj[0,0], trj[0,1], trj[0,2], label="start")
ax.scatter(trj[:,0],trj[:,1],trj[:,2])
ax.scatter(trj[-1,0], trj[-1,1], trj[-1,2], label="end")

plt.legend()
plt.show()

ValueError: Geometric sequence cannot include zero

In [3]:
t1, curve1 = util.artificial_grabing_data(np.array([1,1,0]), np.array([-0.75, -0.75, 0]))
t2, curve2 = util.artificial_grabing_data(np.array([-1, -1, 0]), np.array([0.75, 0.75, 0]))

# np.savetxt("artifical_data/data1.dat", curve1, delimiter='  ')
# np.savetxt("artifical_data/data2.dat", curve2, delimiter='  ')


# print(t1[:,np.newaxis].shape)
# print(curve1.shape)

# exp_data1 = np.hstack((t1[:,np.newaxis],curve1))
# exp_data2 = np.hstack((t2[:,np.newaxis],curve2))

# np.savetxt("artifical_data/data1_gt.dat", exp_data1, delimiter='  ')
# np.savetxt("artifical_data/data2_gt.dat", exp_data2, delimiter='  ')

Normalized et:  [-1.75 -1.75  0.  ]
Normalized et:  [1.75 1.75 0.  ]


In [28]:
curve1 = np.loadtxt("artifical_data/data1_gt.dat")[:,1:]
curve2 = np.loadtxt("artifical_data/data2_gt.dat")[:,1:]

In [29]:
plot_trj_3d([curve1, curve2], ["Player 1", "Player 2"])

In [30]:
from interacting_dmps_david_wode.source_code.parameters import *

In [31]:
collision_data_file_1 = "artifical_data/data1.dat"
collision_data_file_2 = "artifical_data/data2.dat"

In [32]:
startpoint_a1 = curve1[0,:]
startpoint_a2 = curve2[0,:]
endpoint_a1 = curve1[-1,:]
endpoint_a2 = curve2[-1,:]

agent1 = agent(startpoint_a1, endpoint_a1, [1,1,1])
agent2 = agent(startpoint_a2, endpoint_a2, [1,1,1])

In [33]:
agent1.learn_trajectory(collision_data_file_1)
agent2.learn_trajectory(collision_data_file_2)

Start Learning...
|████████████████████████████████████████| 200/200 [100%] in 17.1s (11.73/s) 
Start Learning...
|████████████████████████████████████████| 200/200 [100%] in 17.0s (11.77/s) 


In [34]:
# What happens without direction bias? (but switching back to the approach sensitive function again)
params['interaction_force_params']['direction_bias_const'] = 1e-4
params['interaction_force_params']['func'] = 'heaviside'
params['interaction_force_params']['extrapolate_ego'] = False

# simulate again
# simulation_time = len(collision_data_file_1)
simulation_time = len(collision_data_file_1) * 5
simulate_loosely_coupled_agents([agent1, agent2], T=simulation_time, params=params, delta_t=0.5)

# plot both agent pairs in new plot
plot_agents([agent1, agent2], T=simulation_time, params=params, delta_t=0.5, 
            suptitle=r'No direction bias? -> Collisions!', 
            num_of_interacting_agents=2,
            file_name='collision', 
            list_of_learned_data=[collision_data_file_1, collision_data_file_2],
            plot_pos_at_t= 0.6)
plt.show()